In [1]:
!pip install flask pyngrok PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.1 MB/s eta 0:00:00


In [3]:
!pip install flask pyngrok PyPDF2

import os
from flask import Flask, request, redirect, url_for, send_from_directory, render_template_string
from werkzeug.utils import secure_filename
from pyngrok import ngrok
from PyPDF2 import PdfFileReader, PdfFileWriter

# Initialize Flask app
app = Flask(__name__)

# Configuration
UPLOAD_FOLDER = '/content/uploads'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# HTML Template
html = '''
<!DOCTYPE html>
<html>
<head>
    <title>PDF Password Protect</title>
</head>
<body>
    <h1>Upload PDF to Protect with Password</h1>
    <form method="POST" action="/" enctype="multipart/form-data">
        <label for="file">Choose PDF file:</label>
        <input type="file" name="file" required>
        <br><br>
        <label for="password">Enter Password:</label>
        <input type="password" name="password" required>
        <br><br>
        <button type="submit">Upload and Protect</button>
    </form>
</body>
</html>
'''

@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        # Check if the post request has the file part
        if 'file' not in request.files:
            return 'No file part'
        file = request.files['file']
        if file.filename == '':
            return 'No selected file'
        if file and file.filename.endswith('.pdf'):
            filename = secure_filename(file.filename)
            file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            file.save(file_path)
            password = request.form['password']
            protect_pdf(file_path, password)
            return send_from_directory(app.config['UPLOAD_FOLDER'], 'protected_' + filename, as_attachment=True)
    return render_template_string(html)

def protect_pdf(file_path, password):
    output_pdf = PdfFileWriter()
    input_pdf = PdfFileReader(file_path)

    for i in range(input_pdf.getNumPages()):
        output_pdf.addPage(input_pdf.getPage(i))

    output_pdf.encrypt(password)
    protected_file_path = os.path.join(app.config['UPLOAD_FOLDER'], 'protected_' + os.path.basename(file_path))
    with open(protected_file_path, 'wb') as f:
        output_pdf.write(f)

if __name__ == '__main__':
    # Set ngrok authtoken
    !ngrok authtoken YOUR_NGROK_AUTH_TOKEN

    # Create a public URL
    public_url = ngrok.connect(5000)
    print('Public URL:', public_url)

    # Run Flask app
    app.run(port=5000)


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


ERROR:pyngrok.process.ngrok:t=2024-07-27T15:05:03+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.\nYour authtoken: YOUR_NGROK_AUTH_TOKEN\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n"
ERROR:pyngrok.process.ngrok:t=2024-07-27T15:05:03+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.\nYour authtoken: YOUR_NGROK_AUTH_TOKEN\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n"
ERROR:pyngrok.process.ngrok:t=2024-07-27T15:05:03+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: The authtoken you specified does not look like a pr

PyngrokNgrokError: The ngrok process errored on start: authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.\nYour authtoken: YOUR_NGROK_AUTH_TOKEN\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n.